<a href="https://colab.research.google.com/github/Umapak/masies_tivissasud/blob/gh_viewers/masies_tivissasud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masies de Tivissa Sud

In [1]:
%%capture
!pip install --upgrade gspread
!apt install libspatialindex-dev
!pip install rtree
!pip install geopandas

In [2]:
#@title import from Google drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
# hemen zehaztu behar da zein den google drive-n eman zaion izena
pre1taula = gc.open('CAPLPV').sheet1
# get_all_values gives a list of rows.
erregistroak = pre1taula.get_all_values()
print(erregistroak)

# datu sorta guztia taula erabilgarri formatura pasako du eta campo bakoitzaren izena jarriko du.
import pandas as pd
import geopandas as gpd
dfmasies = pd.DataFrame.from_records(erregistroak)
dfmasies.columns = dfmasies.iloc[0]
dfmasies = dfmasies.iloc[1: , :]

[['id_geo', 'id_incendis', 'cota_m', 'zona_nom', 'masia_nom', 'masia_nom2', 'nombre1_y_telefono1', 'meses_habitados', 'nombre2_y_telefono2', 'telefono1', 'telefono2', 'notas_aclaratorias', 'whatsapp_tiene', 'whatsapp_nombre', 'correo', 'info_adicional', 'etiq_mapa', 'ref_catastral', 'url_mapa', 'url_catastro', 'xetrs89', 'yetrs89', 'latwgs84', 'lonwgs84'], ['1', '', '169', 'Frauques', 'Mas Helios', '', 'Gerard Guez [608 747 852] & Joëlle', '12', '', '34608747852', "'+34 608 747 852", '', '', '608 747 852', '', 'Museo & bodega de vino', '1 Mas Helios (Frauques)', '43152A05300028', 'https://www.instamaps.cat/visor.html?businessid=85fcdf4dd16c1608ef94765154ebc6c7&3D=false#18/40.93459/0.75666', 'https://www1.sedecatastro.gob.es/CYCBienInmueble/OVCConCiud.aspx?del=43&mun=152&UrbRus=&RefC=43152A05300028&Apenom=&esBice=&RCBice1=&RCBice2=&DenoBice=&latitud=&longitud=&gradoslat=&minlat=&seglat=&gradoslon=&minlon=&seglon=&x=&y=&huso=&tipoCoordenadas=&ZV=NO', '311138', '4533919', '40.93459', '0.7

In [3]:
df = dfmasies
# Convert the column to numeric data type
df[['latwgs84','lonwgs84']] = df[['latwgs84', 'lonwgs84']].astype('string')
df['latwgs84'] = df['latwgs84'].str.replace(',', '.')
df['lonwgs84'] = df['lonwgs84'].str.replace(',', '.')
df['latwgs84'] = pd.to_numeric(df['latwgs84'], errors='coerce')
df['lonwgs84'] = pd.to_numeric(df['lonwgs84'], errors='coerce')
df['cota_m'] = pd.to_numeric(df['cota_m'], errors='coerce')
df = df.dropna(subset=['latwgs84', 'lonwgs84'])

In [4]:
# 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 1 to 72
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_geo               72 non-null     object 
 1   id_incendis          72 non-null     object 
 2   cota_m               68 non-null     float64
 3   zona_nom             72 non-null     object 
 4   masia_nom            72 non-null     object 
 5   masia_nom2           72 non-null     object 
 6   nombre1_y_telefono1  72 non-null     object 
 7   meses_habitados      72 non-null     object 
 8   nombre2_y_telefono2  72 non-null     object 
 9   telefono1            72 non-null     object 
 10  telefono2            72 non-null     object 
 11  notas_aclaratorias   72 non-null     object 
 12  whatsapp_tiene       72 non-null     object 
 13  whatsapp_nombre      72 non-null     object 
 14  correo               72 non-null     object 
 15  info_adicional       72 non-null     objec

In [5]:
from shapely.geometry import Point
df.loc[:, 'geometry'] = df.apply(lambda x: Point((float(x.lonwgs84), float(x.latwgs84))), axis=1)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [6]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf = gdf[['masia_nom','whatsapp_tiene','whatsapp_nombre','id_incendis', 'cota_m', 'etiq_mapa', 'ref_catastral', 'geometry','latwgs84', 'lonwgs84']]
gdf.crs = {'init': 'epsg:4326'}
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 72 entries, 1 to 72
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   masia_nom        72 non-null     object  
 1   whatsapp_tiene   72 non-null     object  
 2   whatsapp_nombre  72 non-null     object  
 3   id_incendis      72 non-null     object  
 4   cota_m           68 non-null     float64 
 5   etiq_mapa        72 non-null     object  
 6   ref_catastral    72 non-null     object  
 7   geometry         72 non-null     geometry
 8   latwgs84         72 non-null     float64 
 9   lonwgs84         72 non-null     float64 
dtypes: float64(3), geometry(1), object(6)
memory usage: 6.2+ KB


/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [7]:
import folium
#extraer vértices de marco
sw = gdf[['latwgs84', 'lonwgs84']].min().values.tolist()
ne = gdf[['latwgs84', 'lonwgs84']].max().values.tolist()
#determinar el fondo
tileado_name = "OpenStreetMap" #@param ["Stamen Toner", "Stamen Terrain", "OpenStreetMap", "Cartodbdark_matter"]
#crear el mapa con el fondo elegido
m = folium.Map(tiles=tileado_name)
#poner el marco
m.fit_bounds([sw, ne])

In [8]:
for row in gdf.itertuples():
    folium.Marker([row.geometry.y, row.geometry.x],
                  icon=folium.Icon(
                      color='white', 
                      icon_color='black', 
                      icon='home', 
                      angle=0
                      ),
                  popup = folium.Popup(
                      "<b>{}</b> <br> Id: <b>{}</b> <br> Whats: {} "
                      .format(
                          row.masia_nom, row.id_incendis, row.whatsapp_tiene))).add_to(m)

In [9]:
from google.colab import drive
drive.mount('/content/drive')
m.save("/content/drive/My Drive/AVMBT/geodatos/index.html")

Mounted at /content/drive
